<a href="https://colab.research.google.com/github/ruischnaider/desafio_tera_quinto_andar/blob/main/Desafio_2_Quinto_Andar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Configurações inciais

In [ ]:
import psycopg2
import pandas as pd
import plotly.express as px

def get_data(cursor, tablename):
  postgreSQL_select_Query = 'select * from {tablename}'.format(tablename=tablename)
  
  cursor.execute(postgreSQL_select_Query)
  colnames = [desc[0] for desc in cursor.description]

  return  pd.DataFrame(cursor.fetchall(), columns=colnames)

try:
    connection = psycopg2.connect(user="data-analytics",
                                  password="ExTDDzjzPE7BJ27D",
                                  host="db.candidates.sandbox.data.quintoandar.com.br",
                                  port="5432",
                                  database="quintoandar")
    cursor = connection.cursor()

    # 'tenant_reviews_clean', 'visits_contracts', 'visits_offers_contracts']

    tenant_reviews_clean = get_data(cursor, 'tenant_reviews_clean')
    visits_contracts = get_data(cursor, 'visits_contracts')
    visits_offers_contracts = get_data(cursor, 'visits_offers_contracts')

    print('tenant_reviews_clean', tenant_reviews_clean)
    print('visits_contracts', visits_contracts)
    print('visits_offers_contracts', visits_offers_contracts)

    import datetime

    tenant_reviews_clean['visita_ocorrida'] = tenant_reviews_clean['review'].apply(lambda x: x != 'Visita não aconteceu.')
    tenant_reviews_clean['Mês'] = tenant_reviews_clean['visit_date'].apply(lambda x: pd.to_datetime(x).strftime("%b"))

except (Exception, psycopg2.Error) as error:
    print("Error while fetching data from PostgreSQL", error)

finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")


/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


tenant_reviews_clean        visit_date  ...                                             others
0      2017-12-27  ...                                               None
1      2018-01-03  ...  o zelador muito inflexível. nos deixou pra for...
2      2018-01-03  ...                                               None
3      2018-01-03  ...                                               None
4      2018-01-03  ...                                               None
...           ...  ...                                                ...
13465  2018-08-27  ...                                               None
13466  2018-08-27  ...                                               None
13467  2018-08-27  ...                                               None
13468  2018-08-27  ...                                               None
13469  2018-08-31  ...                              Não possuía 2 quartos

[13470 rows x 18 columns]
visits_contracts     visits  tenants_that_did_not_sign  tenants_

#Exploração

##Volume de visitas x reviews

In [ ]:
def get_month(x):
  switcher = {
      'jan':'Jan',
      'fev':'Feb',
      'mar':'Mar',
      'abr':'Apr',
      'mai':'May',
      'jun':'Jun',
      'jul':'Jul',
      'ago':'Aug',
      'dez':'Dec'
  }
  return switcher.get(x)

visits_offers_contracts['Mês'] = visits_offers_contracts['month'].apply(get_month)
df = visits_offers_contracts[['Mês', 'visits']]
tenant_reviews_clean_g = tenant_reviews_clean.groupby(['Mês']).size().reset_index()
tenant_reviews_clean_g.columns = ['Mês', 'reviews']
df = df.merge(tenant_reviews_clean_g)

df['%'] = df['reviews']/df['visits']

import plotly.graph_objects as go
fig = go.Figure(
    data=[
        go.Bar(name='Visitantes', x=df['Mês'], y=df['visits'], yaxis='y', offsetgroup=1),
        go.Bar(name='Reviews', x=df['Mês'], y=df['reviews'], yaxis='y', offsetgroup=2),
    ],
    layout={
        'yaxis': {'title': 'Visitantes x Reviews'},
        'yaxis2': {'title': '%', 'overlaying': 'y', 'side': 'right'}
    }
)

fig.add_trace(go.Scatter(x=df['Mês'], y=df['%'], name="%", yaxis="y2"))
fig.update_layout(barmode='group', hovermode='x', yaxis2={'tickformat':',.0%'})
fig.show()

##Reviews

In [ ]:
fig = px.pie(tenant_reviews_clean, names='review')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

###Reviews ao longo do tempo

In [ ]:
df = tenant_reviews_clean.groupby(['review', 'Mês']).size().reset_index()
df.columns = ['review', 'Mês', 'count']
df['%'] = df.groupby(['Mês'])['count'].transform(lambda x: x/sum(x))
df
df.reset_index()

fig = px.bar(df, x='review', y='%', text='%', title='Distribuição de reviews', facet_col='Mês', facet_col_wrap=2, category_orders={"Mês":["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug"]})
fig.update_traces(texttemplate='%{text:.2%}')
fig.update_xaxes(type='category', matches='x')
fig.update_yaxes(tickformat=',.2%', matches='y')
fig.show()

df.where((df['review'] == 'Visita não aconteceu.') | (df['review'] == 'EntradaNaoAutorizada'), inplace=True)
df.dropna(inplace=True)
fig = px.bar(df, x='Mês', y='%', text='%', title='Distribuição de reviews - Visitas não ocorridas', category_orders={"Mês":["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug"]})
fig.update_traces(texttemplate='%{text:.2%}')
fig.update_xaxes(type='category', matches='x')
fig.update_yaxes(tickformat=',.2%', matches='y')
fig.show()